<a href="https://www.kaggle.com/code/raunakshrestha007/ppp-rdkit-preprocessing-neuroips?scriptVersionId=255249319" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train.csv
/kaggle/input/neurips-open-polymer-prediction-2025/test.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv
/kaggle/input/tc-smiles/Tc_SMILES.csv
/kaggle/input/tg-smiles-pid-polymer-class/TgSS_enriched_cleaned.csv
/kaggle/input/smiles-extra-data/data_dnst1.xlsx
/kaggle/input/smiles-extra-data/data_tg3.xlsx
/kaggle/input/smiles-extra-data/JCIM_sup_bigsmiles.csv
/kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl


In [2]:
!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl

Processing /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl


# Data Loading and Preprocessing Pipeline

In [3]:
# === Imports ===
import pandas as pd
import numpy as np
from rdkit import Chem

# === Config ===
BASE_PATH = '/kaggle/input/neurips-open-polymer-prediction-2025/'
TARGETS = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
BAD_PATTERNS = ['[R]', '[R1]', '[R2]', '[R3]', '[R4]', '[R5]',
                "[R']", '[R"]', 'R1', 'R2', 'R3', 'R4', 'R5',
                '([R])', '([R1])', '([R2])']

# === SMILES Cleaner ===
def clean_and_validate_smiles(smiles):
    if not isinstance(smiles, str) or not smiles:
        return None
    for pattern in BAD_PATTERNS:
        if pattern in smiles:
            return None
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            return Chem.MolToSmiles(mol, canonical=True)
    except:
        return None
    return None

# === Load Train/Test ===
train = pd.read_csv(BASE_PATH + 'train.csv')
test = pd.read_csv(BASE_PATH + 'test.csv')

train['SMILES'] = train['SMILES'].apply(clean_and_validate_smiles)
test['SMILES'] = test['SMILES'].apply(clean_and_validate_smiles)

train.dropna(subset=['SMILES'], inplace=True)
test.dropna(subset=['SMILES'], inplace=True)

# === Load External Datasets (excluding dataset2) ===
external_datasets = []

def load_external(path, target, rename_map=None):
    try:
        df = pd.read_csv(path)
        if rename_map:
            df = df.rename(columns=rename_map)
        if 'SMILES' in df.columns and target in df.columns:
            df = df[['SMILES', target]].dropna()
            external_datasets.append((target, df))
            print(f"✅ Loaded {path} ({len(df)} entries for {target})")
        else:
            print(f"⚠️ Skipped {path}: required columns missing")
    except Exception as e:
        print(f"⚠️ Failed to load {path}: {e}")

load_external(BASE_PATH + 'train_supplement/dataset1.csv', 'Tc', rename_map={'TC_mean': 'Tc'})
load_external(BASE_PATH + 'train_supplement/dataset3.csv', 'Tg')
load_external(BASE_PATH + 'train_supplement/dataset4.csv', 'FFV')

# === Load Additional External Datasets ===
try:
    extra_data_tg3 = pd.read_excel("/kaggle/input/smiles-extra-data/data_tg3.xlsx")
    extra_data_dnst1 = pd.read_excel("/kaggle/input/smiles-extra-data/data_dnst1.xlsx")
    jcim_sup_bigsmiles = pd.read_csv("/kaggle/input/smiles-extra-data/JCIM_sup_bigsmiles.csv")
    tc_smiles_df = pd.read_csv("/kaggle/input/tc-smiles/Tc_SMILES.csv")
except Exception as e:
    print(f"⚠️ Error loading extra data: {e}")

# Helper to standardize and append
def process_and_append_external(df, target, source_name):
    if 'SMILES' in df.columns and target in df.columns:
        df = df[['SMILES', target]].copy()
        df['SMILES'] = df['SMILES'].apply(clean_and_validate_smiles)
        df = df.dropna(subset=['SMILES'])

        # Ensure the target column is numeric
        df[target] = pd.to_numeric(df[target], errors='coerce')
        df = df.dropna(subset=[target])

        df = df.groupby('SMILES', as_index=False)[target].mean()
        external_datasets.append((target, df))
        print(f"✅ Integrated {source_name}: {len(df)} entries for {target}")
    else:
        print(f"⚠️ Skipped {source_name}: missing columns")

# Process each extra dataset (with correct column names)
process_and_append_external(extra_data_tg3.rename(columns={"Tg [K]": "Tg"}), "Tg", "data_tg3.xlsx")
process_and_append_external(extra_data_dnst1.rename(columns={"density(g/cm3)": "Density"}), "Density", "data_dnst1.xlsx")
process_and_append_external(tc_smiles_df.rename(columns={"TC_mean": "Tc"}), "Tc", "Tc_SMILES.csv")

# JCIM SMILES only (for future feature engineering)
jcim_smiles_only = jcim_sup_bigsmiles[['SMILES']].dropna()
jcim_smiles_only['SMILES'] = jcim_smiles_only['SMILES'].apply(clean_and_validate_smiles)
jcim_smiles_only = jcim_smiles_only.dropna().drop_duplicates()
print(f"✅ Loaded JCIM SMILES-only dataset: {len(jcim_smiles_only)} unique SMILES (no targets)")

# === Merge External Data ===
def merge_external(train_df, ext_df, target):
    ext_df['SMILES'] = ext_df['SMILES'].apply(clean_and_validate_smiles)
    ext_df = ext_df.dropna(subset=['SMILES', target])
    ext_df = ext_df.groupby('SMILES', as_index=False)[target].mean()

    # Fill missing target values in existing rows
    existing_smiles = set(train_df['SMILES'])
    to_fill = ext_df[ext_df['SMILES'].isin(existing_smiles)]
    for _, row in to_fill.iterrows():
        mask = (train_df['SMILES'] == row['SMILES']) & (train_df[target].isna())
        train_df.loc[mask, target] = row[target]

    # Add new rows
    new_smiles = set(ext_df['SMILES']) - existing_smiles
    new_rows = ext_df[ext_df['SMILES'].isin(new_smiles)].copy()
    for col in TARGETS:
        if col not in new_rows.columns:
            new_rows[col] = np.nan
    return pd.concat([train_df, new_rows[['SMILES'] + TARGETS]], ignore_index=True)

# === Apply Merges ===
train_extended = train[['SMILES'] + TARGETS].copy()
for target, ext in external_datasets:
    train_extended = merge_external(train_extended, ext, target)

# === Final Clean-Up ===
train_extended = train_extended.replace([np.inf, -np.inf], np.nan)
train_extended = train_extended.dropna(subset=TARGETS, how='all')
train_extended = train_extended.drop_duplicates(subset=['SMILES']).reset_index(drop=True)

# === Summary ===
print("\n📊 Final Summary:")
print(f"Train: {len(train)} | Extended: {len(train_extended)}")
for t in TARGETS:
    base = train[t].notna().sum()
    ext = train_extended[t].notna().sum()
    print(f"• {t:<8}: {ext} total ({ext - base:+} from supplements)")

print("\n✅ Data loading and preprocessing complete.")

smiles_list = train_extended['SMILES'].tolist()
# Clean SMILES column robustly
train_extended['SMILES'] = train_extended['SMILES'].apply(clean_and_validate_smiles)
# === Final Clean-Up ===
train_extended = train_extended.replace([np.inf, -np.inf], np.nan)
train_extended = train_extended.dropna(subset=TARGETS, how='all')
train_extended = train_extended.drop_duplicates(subset=['SMILES']).reset_index(drop=True)

# === Drop constant columns ===
constant_cols = [col for col in train_extended.columns if train_extended[col].nunique() == 1]
train_extended.drop(columns=constant_cols, inplace=True)
print(f"Dropped {len(constant_cols)} constant columns from train_extended")


train_extended.shape
train_extended


✅ Loaded /kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv (874 entries for Tc)
✅ Loaded /kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv (46 entries for Tg)
✅ Loaded /kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv (862 entries for FFV)
✅ Integrated data_tg3.xlsx: 499 entries for Tg
✅ Integrated data_dnst1.xlsx: 778 entries for Density
✅ Integrated Tc_SMILES.csv: 866 entries for Tc
✅ Loaded JCIM SMILES-only dataset: 662 unique SMILES (no targets)

📊 Final Summary:
Train: 7973 | Extended: 9990
• Tg      : 1056 total (+545 from supplements)
• FFV     : 7892 total (+862 from supplements)
• Tc      : 866 total (+129 from supplements)
• Density : 1247 total (+634 from supplements)
• Rg      : 614 total (+0 from supplements)

✅ Data loading and preprocessing complete.
Dropped 0 constant columns from train_extended


,SMILES,Tg,FFV,Tc,Density,Rg
0,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,1.05,NaN
1,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN
2,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN
3,*Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)...,NaN,0.387324,NaN,NaN,NaN
4,*Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...,NaN,0.355470,NaN,NaN,NaN
...,...,...,...,...,...,...
9985,c1ccc(-c2ccccn2)nc1,NaN,NaN,NaN,1.31,NaN
9986,c1ccc(-c2nc3cc4ncoc4cc3o2)cc1,NaN,NaN,NaN,1.43,NaN
9987,c1ccc2oc(-c3ccc4ncoc4c3)nc2c1,NaN,NaN,NaN,1.43,NaN
9988,c1ccsc1,NaN,NaN,NaN,1.51,NaN


# Preprocessing Each Property Separately

In [4]:
from rdkit.Chem import AllChem, Descriptors
from sklearn.preprocessing import MinMaxScaler
from rdkit.Chem import rdDistGeom
import random
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# === Tg: Glass Transition Temperature Preprocessing ===
def preprocess_tg(df):
    print(f"[Tg] Starting with {len(df)} unique SMILES")
    df = df.drop_duplicates(subset='SMILES').copy()
    df['mol'] = df['SMILES'].apply(lambda s: Chem.MolFromSmiles(s, sanitize=False))

    # Sanitize molecules but allow wildcard '*' (polymerization points)
    def is_valid_mol(mol):
        try:
            if mol is None:
                return False
            Chem.SanitizeMol(mol, catchErrors=True)
            return True
        except Exception as e:
            return False

    df['is_valid'] = df['mol'].apply(is_valid_mol)
    invalid_count = (~df['is_valid']).sum()
    print(f"[Tg] Invalid mols removed (e.g. parsing/sanitization issues): {invalid_count}")
    df = df[df['is_valid']].drop(columns='is_valid')

    # Generate 3D conformers
    def generate_conformers(mol):
        try:
            mol = Chem.AddHs(mol)
            params = AllChem.ETKDGv3()
            params.randomSeed = 42
            ids = AllChem.EmbedMultipleConfs(mol, numConfs=3, params=params)
            return ids if ids else []
        except:
            return []

    df['conformers'] = df['mol'].apply(generate_conformers)
    no_conf = (df['conformers'].apply(len) == 0).sum()
    print(f"[Tg] Molecules with 0 conformers: {no_conf}")
    df = df[df['conformers'].apply(len) > 0]

    print(f"[Tg] Final Tg samples: {len(df)}")
    return df


# === FFV: Fractional Free Volume Preprocessing ===
def preprocess_ffv(df):
    df = df.copy()
    # Remove invalid structures and extreme outliers
    df = df[df['FFV'].between(0.0, 1.0)]  # plausible physical bounds
    return df

# === Tc: Thermal Conductivity Preprocessing ===
def preprocess_tc(df):
    df = df.copy()
    # Remove noise: remove outliers using IQR
    q1 = df['Tc'].quantile(0.25)
    q3 = df['Tc'].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    df = df[df['Tc'].between(lower, upper)]

    # Data augmentation (e.g., duplicate underrepresented values)
    # Here: oversample rare Tc ranges (<0.1, >1.5)
    low_samples = df[df['Tc'] < 0.1]
    high_samples = df[df['Tc'] > 1.5]
    df = pd.concat([df, low_samples, high_samples], ignore_index=True)

    # Normalize with MinMax
    scaler = MinMaxScaler()
    df['Tc_scaled'] = scaler.fit_transform(df[['Tc']])
    return df

# === Density: Polymer Density Preprocessing ===
def preprocess_density(df, cap_per_bin=300):
    print(f"[Density] Starting with {len(df)} samples")
    df = df.copy()

    # Step 1: Filter valid physical density range
    df = df[df['Density'].between(0.5, 2.0)]
    print(f"[Density] After bounds filter (0.5–2.0): {len(df)} samples")

    # Step 2: Normalize SMILES (canonicalize & validate)
    def normalize_smiles(smi):
        try:
            mol = Chem.MolFromSmiles(smi)
            if mol is None:
                return None
            return Chem.MolToSmiles(mol, canonical=True)
        except:
            return None

    df['SMILES_norm'] = df['SMILES'].apply(normalize_smiles)
    num_invalid = df['SMILES_norm'].isnull().sum()
    print(f"[Density] Invalid SMILES removed during normalization: {num_invalid}")

    df = df[df['SMILES_norm'].notnull()].copy()
    df['SMILES'] = df['SMILES_norm']
    df = df.drop(columns='SMILES_norm')

    # Step 3: Bin densities
    df['density_bin'] = pd.cut(df['Density'], bins=[0.5, 1.0, 1.5, 2.0])
    bin_counts = df['density_bin'].value_counts().sort_index()
    print("[Density] Bin counts before capping:")
    print(bin_counts)

    # Step 4: Cap each bin to avoid large imbalance
    def cap_bin(group):
        return group.sample(min(len(group), cap_per_bin), random_state=42)

    df = (
        df.groupby('density_bin', observed=False)
        .apply(cap_bin)
        .reset_index(drop=True)
    )

    print(f"[Density] Final capped samples: {len(df)}")
    return df


# === Rg: Radius of Gyration Preprocessing ===
def preprocess_rg(df):
    from rdkit import Chem
    from rdkit.Chem import AllChem

    df = df.copy()

    def smiles_to_3d_polymer_safe(smiles):
        try:
            # Parse SMILES without sanitizing (to allow wildcard *)
            mol = Chem.MolFromSmiles(smiles, sanitize=False)
            if mol is None:
                return None

            # Manually update valence information (avoids implicit Hs error)
            for atom in mol.GetAtoms():
                atom.UpdatePropertyCache(strict=False)

            # Add explicit hydrogens
            mol = Chem.AddHs(mol)

            # Generate 3D conformer
            params = AllChem.ETKDGv3()
            params.randomSeed = 42
            success = AllChem.EmbedMolecule(mol, params)
            if success != 0:
                return None

            # Optimize geometry
            AllChem.UFFOptimizeMolecule(mol)

            return mol

        except Exception as e:
            return None

    # Apply 3D generation to Rg SMILES
    df['mol_3d'] = df['SMILES'].apply(smiles_to_3d_polymer_safe)

    # Drop failed conversions
    df = df[df['mol_3d'].notnull()].reset_index(drop=True)

    return df



# === Apply All Preprocessing Steps ===
tg_df = preprocess_tg(train_extended[train_extended['Tg'].notna()])
ffv_df = preprocess_ffv(train_extended[train_extended['FFV'].notna()])
tc_df = preprocess_tc(train_extended[train_extended['Tc'].notna()])
density_df = preprocess_density(train_extended[train_extended['Density'].notna()], cap_per_bin=300)
rg_df = preprocess_rg(train_extended[train_extended['Rg'].notna()])

print("✅ All property-specific preprocessing complete.")
print(f"Tg samples: {len(tg_df)}")
print(f"FFV samples: {len(ffv_df)}")
print(f"Tc samples: {len(tc_df)}")
print(f"Density samples: {len(density_df)}")
print(f"Rg samples: {len(rg_df)}")


[Tg] Starting with 1056 unique SMILES
[Tg] Invalid mols removed (e.g. parsing/sanitization issues): 0
[Tg] Molecules with 0 conformers: 15
[Tg] Final Tg samples: 1041
[Density] Starting with 1247 samples
[Density] After bounds filter (0.5–2.0): 1246 samples
[Density] Invalid SMILES removed during normalization: 0
[Density] Bin counts before capping:
density_bin
(0.5, 1.0]    456
(1.0, 1.5]    673
(1.5, 2.0]    117
Name: count, dtype: int64
[Density] Final capped samples: 717
✅ All property-specific preprocessing complete.
Tg samples: 1041
FFV samples: 7892
Tc samples: 879
Density samples: 717
Rg samples: 597


# Feature Engineering Pipeline

In [5]:
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem
import numpy as np
import pandas as pd
from tqdm import tqdm

# === Tg Feature Engineering ===
def featurize_tg(df):
    df = df.copy()
    df['mol'] = df['SMILES'].apply(Chem.MolFromSmiles)

    # Morgan fingerprints (radius=2, nBits=1024)
    df['MorganFP'] = df['mol'].apply(lambda m: AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=1024) if m else None)

    # Physico-chemical descriptors
    desc_funcs = {
        'MolWt': Descriptors.MolWt,
        'NumHDonors': Descriptors.NumHDonors,
        'NumHAcceptors': Descriptors.NumHAcceptors,
        'TPSA': rdMolDescriptors.CalcTPSA,
        'MolLogP': Descriptors.MolLogP,
        'NumRotatableBonds': Descriptors.NumRotatableBonds,
    }
    
    feats = {name: [] for name in desc_funcs}
    for m in tqdm(df['mol'], desc='Featurizing Tg'):
        if m:
            for name, func in desc_funcs.items():
                feats[name].append(func(m))
        else:
            for name in desc_funcs:
                feats[name].append(np.nan)
    
    feats_df = pd.DataFrame(feats, index=df.index)
    df = pd.concat([df, feats_df], axis=1)

    return df

# === FFV Feature Engineering ===
def featurize_ffv(df):
    df = df.copy()
    df['mol'] = df['SMILES'].apply(Chem.MolFromSmiles)

    feats = {
        'MolWt': [],
        'TPSA': [],
        'LabuteASA': [],
        'PEOE_VSA1': [],
    }
    for m in tqdm(df['mol'], desc='Featurizing FFV'):
        if m:
            feats['MolWt'].append(Descriptors.MolWt(m))
            feats['TPSA'].append(rdMolDescriptors.CalcTPSA(m))
            feats['LabuteASA'].append(rdMolDescriptors.CalcLabuteASA(m))
            feats['PEOE_VSA1'].append(Descriptors.PEOE_VSA1(m)) 
        else:
            for k in feats:
                feats[k].append(np.nan)

    feats_df = pd.DataFrame(feats, index=df.index)
    df = pd.concat([df, feats_df], axis=1)
    return df

# === Tc Feature Engineering ===
def featurize_tc(df):
    df = df.copy()
    df['mol'] = df['SMILES'].apply(Chem.MolFromSmiles)
    
    feats = {
        'MolSurfaceArea': [],
        'MolVolume': [],
        'NumAtoms': [],
        'InteratomicDistancesMean': [],
    }

    for m in tqdm(df['mol'], desc='Featurizing Tc'):
        if m:
            # Molecular surface area & volume using Crippen descriptors as proxy
            feats['MolSurfaceArea'].append(Descriptors.MolMR(m))
            feats['MolVolume'].append(Descriptors.MolLogP(m))  # substitute for volume, or replaceww with 3D volume if available
            feats['NumAtoms'].append(m.GetNumAtoms())
            
            # Mean pairwise interatomic distances (from 3D conf if present)
            try:
                mol3d = Chem.AddHs(m)
                AllChem.EmbedMolecule(mol3d)
                conf = mol3d.GetConformer()
                dists = []
                n_atoms = mol3d.GetNumAtoms()
                for i in range(n_atoms):
                    pos_i = conf.GetAtomPosition(i)
                    for j in range(i+1, n_atoms):
                        pos_j = conf.GetAtomPosition(j)
                        dist = pos_i.Distance(pos_j)
                        dists.append(dist)
                feats['InteratomicDistancesMean'].append(np.mean(dists) if dists else np.nan)
            except:
                feats['InteratomicDistancesMean'].append(np.nan)
        else:
            for k in feats:
                feats[k].append(np.nan)
    
    feats_df = pd.DataFrame(feats, index=df.index)
    df = pd.concat([df, feats_df], axis=1)
    return df

# === Density Feature Engineering ===
def featurize_density(df):
    df = df.copy()
    df['mol'] = df['SMILES'].apply(Chem.MolFromSmiles)
    
    feats = {
        'MolVolume3D': [],
        'MolSurfaceArea3D': [],
        'Density_MD': [],
    }

    for m in tqdm(df['mol'], desc='Featurizing Density'):
        if m:
            try:
                mol3d = Chem.AddHs(m)
                AllChem.EmbedMolecule(mol3d, randomSeed=42)
                AllChem.UFFOptimizeMolecule(mol3d)
                vol = AllChem.ComputeMolVolume(mol3d)
                sa = AllChem.ComputeMolSurfaceArea(mol3d)
                feats['MolVolume3D'].append(vol)
                feats['MolSurfaceArea3D'].append(sa)
                feats['Density_MD'].append(np.nan)  # replace with MD density if available
            except:
                feats['MolVolume3D'].append(np.nan)
                feats['MolSurfaceArea3D'].append(np.nan)
                feats['Density_MD'].append(np.nan)
        else:
            for k in feats:
                feats[k].append(np.nan)
    
    feats_df = pd.DataFrame(feats, index=df.index)
    df = pd.concat([df, feats_df], axis=1)
    return df

# === Rg Feature Engineering ===
def featurize_rg(df):
    df = df.copy()
    df['mol_3d'] = df['SMILES'].apply(lambda smi: Chem.MolFromSmiles(smi))
    
    feats = {
        'Radius': [],
        'MomentOfInertia': [],
        'ConformerCoordsMeanX': [],
        'ConformerCoordsMeanY': [],
        'ConformerCoordsMeanZ': [],
    }
    
    for m in tqdm(df['mol_3d'], desc='Featurizing Rg'):
        if m:
            try:
                mol3d = Chem.AddHs(m)
                AllChem.EmbedMolecule(mol3d, randomSeed=42)
                AllChem.UFFOptimizeMolecule(mol3d)
                conf = mol3d.GetConformer()
                n_atoms = mol3d.GetNumAtoms()
                coords = np.array([list(conf.GetAtomPosition(i)) for i in range(n_atoms)])
                
                center = coords.mean(axis=0)
                dists = np.linalg.norm(coords - center, axis=1)
                radius = dists.max()
                
                mass = np.array([atom.GetMass() for atom in mol3d.GetAtoms()])
                rel_coords = coords - center
                moi = np.sum(mass * np.sum(rel_coords**2, axis=1))
                
                feats['Radius'].append(radius)
                feats['MomentOfInertia'].append(moi)
                feats['ConformerCoordsMeanX'].append(center[0])
                feats['ConformerCoordsMeanY'].append(center[1])
                feats['ConformerCoordsMeanZ'].append(center[2])
            except:
                for k in feats:
                    feats[k].append(np.nan)
        else:
            for k in feats:
                feats[k].append(np.nan)
    
    feats_df = pd.DataFrame(feats, index=df.index)
    df = pd.concat([df, feats_df], axis=1)
    return df


# === Example usage ===
tg_features = featurize_tg(tg_df)
ffv_features = featurize_ffv(ffv_df)
tc_features = featurize_tc(tc_df)
density_features = featurize_density(density_df)
rg_features = featurize_rg(rg_df)

print("Feature engineering complete.")
print(f"Tg features shape: {tg_features.shape}")
print(f"FFV features shape: {ffv_features.shape}")
print(f"Tc features shape: {tc_features.shape}")
print(f"Density features shape: {density_features.shape}")
print(f"Rg features shape: {rg_features.shape}")


Featurizing Rg: 100%|██████████| 597/597 [01:03<00:00,  9.39it/s]

Feature engineering complete.
Tg features shape: (1041, 15)
FFV features shape: (7892, 11)
Tc features shape: (879, 12)
Density features shape: (717, 11)
Rg features shape: (597, 12)


# FFV

# CatBoost Pipeline for FFV

In [6]:
import pandas as pd

# Load test data
test_df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')

# Make sure 'SMILES' column exists in test_df (it should, based on challenge format)
print(f"Test data shape: {test_df.shape}")
print(test_df.head())

# Extract test IDs for submission later
test_ids = test_df['id']


Test data shape: (3, 2)
           id                                             SMILES
0  1109053969  *Oc1ccc(C=NN=Cc2ccc(Oc3ccc(C(c4ccc(*)cc4)(C(F)...
1  1422188626  *Oc1ccc(C(C)(C)c2ccc(Oc3ccc(C(=O)c4cccc(C(=O)c...
2  2032016830  *c1cccc(OCCCCCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6...


In [7]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import pandas as pd
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import AllChem

# ====== Step 1: Preprocess test data for FFV =====
def preprocess_ffv(df):
    df = df.copy()
    return df

# ====== Step 2: Feature engineering for test data =====
def featurize_ffv(df):
    df = df.copy()
    feats = {
        'MolWt': [],
        'TPSA': [],
        'NumRotatableBonds': [],
        'MolLogP': [],
        'NumHAcceptors': [],
        'NumHDonors': [],
        'TopologicalPolarSurfaceArea': [],
        'HeavyAtomCount': [],
        'FractionCSP3': [],
        'NumValenceElectrons': [],
        'NumAliphaticRings': [],
    }
    for smi in tqdm(df['SMILES'], desc='Featurizing FFV test'):
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            for k in feats:
                feats[k].append(np.nan)
            continue
        try:
            feats['MolWt'].append(Descriptors.MolWt(mol))
            feats['TPSA'].append(Descriptors.TPSA(mol))
            feats['NumRotatableBonds'].append(Descriptors.NumRotatableBonds(mol))
            feats['MolLogP'].append(Descriptors.MolLogP(mol))
            feats['NumHAcceptors'].append(Descriptors.NumHAcceptors(mol))
            feats['NumHDonors'].append(Descriptors.NumHDonors(mol))
            feats['TopologicalPolarSurfaceArea'].append(Descriptors.TPSA(mol))
            feats['HeavyAtomCount'].append(Descriptors.HeavyAtomCount(mol))
            feats['FractionCSP3'].append(Descriptors.FractionCSP3(mol))
            feats['NumValenceElectrons'].append(Descriptors.NumValenceElectrons(mol))
            feats['NumAliphaticRings'].append(Descriptors.NumAliphaticRings(mol))
        except:
            for k in feats:
                feats[k].append(np.nan)
    feats_df = pd.DataFrame(feats, index=df.index)
    df = pd.concat([df, feats_df], axis=1)
    df = df.drop(columns=['SMILES'])
    return df

# ====== Step 3: Prepare training data =====
ffv_df = preprocess_ffv(train_extended[train_extended['FFV'].notna()])
ffv_features = featurize_ffv(ffv_df)
ffv_features['FFV'] = ffv_df['FFV'].values

# ====== Step 4: Prepare test data features =====
test_preprocessed = preprocess_ffv(test_df)  # Apply same preprocessing on test SMILES
ffv_test_features = featurize_ffv(test_preprocessed)

# ====== Step 5: Align test features to train columns =====
X = ffv_features.drop(columns=['FFV'])
y = ffv_features['FFV']

missing_cols = set(X.columns) - set(ffv_test_features.columns)
for col in missing_cols:
    ffv_test_features[col] = 0
ffv_test_features = ffv_test_features[X.columns]

test_X = ffv_test_features.copy()
test_ids = test_df['id'].copy()

# ====== Step 6: Define evaluation metric =====
def evaluate(y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mape, r2

# ====== Step 7: CatBoost model with KFold CV =====
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(test_X))

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n🔁 Fold {fold+1}")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    train_pool = Pool(X_train, y_train)
    val_pool = Pool(X_val, y_val)

    model = CatBoostRegressor(
        iterations=5000,
        learning_rate=0.03,
        depth=6,
        l2_leaf_reg=3,
        loss_function='RMSE',
        random_seed=42,
        early_stopping_rounds=100,
        verbose=100
    )

    model.fit(train_pool, eval_set=val_pool)
    val_pred = model.predict(X_val)
    oof_preds[val_idx] = val_pred
    test_preds += model.predict(test_X) / kf.n_splits

    rmse, mape, r2 = evaluate(y_val, val_pred)
    print(f"✅ RMSE: {rmse:.4f}, MAPE: {mape:.4f}, R²: {r2:.4f}")

# ====== Step 8: Final CV performance =====
rmse, mape, r2 = evaluate(y, oof_preds)
print(f"\n📊 Overall CV Performance — RMSE: {rmse:.4f}, MAPE: {mape:.4f}, R²: {r2:.4f}")

# ====== Step 9: Save submission =====
submission = pd.DataFrame({'id': test_ids, 'FFV': test_preds})
submission.to_csv('ffv_catboost_submission.csv', index=False)
print("✅ Submission saved to ffv_catboost_submission.csv")


Featurizing FFV test: 100%|██████████| 3/3 [00:00<00:00, 108.14it/s]



🔁 Fold 1
0:	learn: 0.0287358	test: 0.0289863	best: 0.0289863 (0)	total: 59.8ms	remaining: 4m 58s
100:	learn: 0.0206618	test: 0.0215988	best: 0.0215988 (100)	total: 494ms	remaining: 23.9s
200:	learn: 0.0188095	test: 0.0206359	best: 0.0206359 (200)	total: 855ms	remaining: 20.4s
300:	learn: 0.0177128	test: 0.0201280	best: 0.0201280 (300)	total: 1.29s	remaining: 20.1s
400:	learn: 0.0168511	test: 0.0198326	best: 0.0198326 (400)	total: 1.66s	remaining: 19.1s
500:	learn: 0.0160286	test: 0.0195218	best: 0.0195218 (500)	total: 1.94s	remaining: 17.5s
600:	learn: 0.0154107	test: 0.0193369	best: 0.0193369 (600)	total: 2.22s	remaining: 16.3s
700:	learn: 0.0148584	test: 0.0191754	best: 0.0191754 (700)	total: 2.5s	remaining: 15.3s
800:	learn: 0.0144031	test: 0.0190576	best: 0.0190576 (800)	total: 2.78s	remaining: 14.6s
900:	learn: 0.0140075	test: 0.0189579	best: 0.0189579 (900)	total: 3.06s	remaining: 13.9s
1000:	learn: 0.0136581	test: 0.0188901	best: 0.0188901 (1000)	total: 3.34s	remaining: 13.4s
1

# GNN Pipeline for FFV

In [8]:
!pip uninstall -y torch torch-scatter torch-sparse torch-geometric

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124


In [9]:
!pip install torch==2.1.0+cpu torchvision==0.16.0+cpu torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.9/184.9 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.8 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.6.0+cu124
    Uninstalling torchaudio-2.6.0+cu124:
      Successfully uninstalled torchaudio-2.6.0+cu124
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.21.0+cu124
    Uninstalling torchvision-0.21.0+cu124:
      Successfully uninstalled torchvision-0.21.0+cu124


In [10]:
!pip install torch-scatter torch-sparse torch-geometric -f https://data.pyg.org/whl/torch-2.1.0+cpu.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.4/500.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.0 MB/s eta 0:00:00


In [11]:
import torch
import torch_geometric
print(torch.__version__)
print(torch_geometric.__version__)

2.1.0+cpu
2.6.1


In [12]:
# ✅ Full GNN Pipeline for FFV Prediction on Kaggle
# Make sure to install torch-geometric before running this (see instructions below)

# Step 0: Install torch-geometric (run this in a separate cell first)
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+cpu.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-2.1.0+cpu.html
# !pip install torch-geometric

import os
import pandas as pd
import numpy as np
from rdkit import Chem
from tqdm import tqdm

import torch
from torch.nn import Linear, ReLU, Dropout, Sequential
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

# ===================== Load Data =====================
train_df = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train.csv")
test_df = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/test.csv")

# ===================== Helper: SMILES to Graph =====================
from rdkit.Chem import rdmolops

def mol_to_graph(smiles, target=None):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    atom_feats = []
    for atom in mol.GetAtoms():
        atom_feats.append([atom.GetAtomicNum()])

    edge_index = [[], []]
    for bond in mol.GetBonds():
        start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        edge_index[0] += [start, end]
        edge_index[1] += [end, start]

    x = torch.tensor(atom_feats, dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long)

    data = Data(x=x, edge_index=edge_index)
    if target is not None:
        data.y = torch.tensor([target], dtype=torch.float)
    return data

# ===================== Prepare Graph Data =====================
train_df_ffv = train_df[train_df['FFV'].notna()].reset_index(drop=True)

train_graphs = []
for i, row in tqdm(train_df_ffv.iterrows(), total=len(train_df_ffv), desc='Processing Train Graphs'):
    data = mol_to_graph(row['SMILES'], row['FFV'])
    if data:
        train_graphs.append(data)

# ===================== GNN Model =====================
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(1, 64)
        self.conv2 = GCNConv(64, 128)
        self.fc = Sequential(
            Linear(128, 64),
            ReLU(),
            Dropout(0.2),
            Linear(64, 1)
        )

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        out = self.fc(x)
        return out.squeeze()

# ===================== Training Loop =====================
def evaluate(y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mape, r2

kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(train_graphs))

for fold, (train_idx, val_idx) in enumerate(kf.split(train_graphs)):
    print(f"\n🔁 Fold {fold+1}")
    train_subset = [train_graphs[i] for i in train_idx]
    val_subset = [train_graphs[i] for i in val_idx]

    train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)

    model = GCN().to('cuda' if torch.cuda.is_available() else 'cpu')
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = torch.nn.MSELoss()

    best_val_rmse = float('inf')
    for epoch in range(1, 201):
        model.train()
        for batch in train_loader:
            batch = batch.to('cuda' if torch.cuda.is_available() else 'cpu')
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index, batch.batch)
            loss = loss_fn(out, batch.y.view(-1))
            loss.backward()
            optimizer.step()

        model.eval()
        val_preds, val_trues = [], []
        for batch in val_loader:
            batch = batch.to('cuda' if torch.cuda.is_available() else 'cpu')
            with torch.no_grad():
                out = model(batch.x, batch.edge_index, batch.batch)
            val_preds.extend(out.cpu().numpy())
            val_trues.extend(batch.y.view(-1).cpu().numpy())

        rmse, mape, r2 = evaluate(val_trues, val_preds)
        if epoch % 20 == 0:
            print(f"Epoch {epoch:3d} | Val RMSE: {rmse:.4f} | R²: {r2:.4f}")

        if rmse < best_val_rmse:
            best_val_rmse = rmse
            oof_preds[val_idx] = val_preds

# ===================== Final Evaluation =====================
y_true = [g.y.item() for g in train_graphs]
rmse, mape, r2 = evaluate(y_true, oof_preds)
print(f"\n📊 GNN CV Performance — RMSE: {rmse:.4f}, MAPE: {mape:.4f}, R²: {r2:.4f}")


Processing Train Graphs: 100%|██████████| 7030/7030 [00:06<00:00, 1030.19it/s]



🔁 Fold 1
Epoch  20 | Val RMSE: 0.0322 | R²: -0.0039
Epoch  40 | Val RMSE: 0.0322 | R²: -0.0038
Epoch  60 | Val RMSE: 0.0322 | R²: -0.0026
Epoch  80 | Val RMSE: 0.0322 | R²: -0.0030
Epoch 100 | Val RMSE: 0.0322 | R²: -0.0006
Epoch 120 | Val RMSE: 0.0322 | R²: -0.0010
Epoch 140 | Val RMSE: 0.0322 | R²: -0.0013
Epoch 160 | Val RMSE: 0.0322 | R²: -0.0012
Epoch 180 | Val RMSE: 0.0322 | R²: -0.0037
Epoch 200 | Val RMSE: 0.0322 | R²: -0.0001

🔁 Fold 2
Epoch  20 | Val RMSE: 0.0299 | R²: -0.0458
Epoch  40 | Val RMSE: 0.0296 | R²: -0.0258
Epoch  60 | Val RMSE: 0.0291 | R²: 0.0096
Epoch  80 | Val RMSE: 0.0292 | R²: 0.0010
Epoch 100 | Val RMSE: 0.0294 | R²: -0.0118
Epoch 120 | Val RMSE: 0.0292 | R²: -0.0018
Epoch 140 | Val RMSE: 0.0293 | R²: -0.0059
Epoch 160 | Val RMSE: 0.0290 | R²: 0.0127
Epoch 180 | Val RMSE: 0.0287 | R²: 0.0380
Epoch 200 | Val RMSE: 0.0287 | R²: 0.0381

🔁 Fold 3
Epoch  20 | Val RMSE: 0.0281 | R²: -0.0151
Epoch  40 | Val RMSE: 0.0280 | R²: -0.0107
Epoch  60 | Val RMSE: 0.0277 